In [44]:
import pdfplumber
import re
import json

def gerar_resumo(dados: dict) -> str:
    """
    Gera um resumo em linguagem natural a partir dos dados estruturados.
    """
    if not dados["numero_matricula"]:
        return "Não foi possível gerar o resumo. Matrícula não encontrada."

    # Determina o proprietário atual
    proprietario_atual = "Não identificado"
    data_consolidacao = ""
    for ato in reversed(dados["historico_atos"]):
        if ato["tipo"] == "Consolidação de Propriedade":
            match_prop = re.search(r'em favor d[oa] credor[ae]\s+e\s+proprietári[oa]\s+fiduciári[oa]:\s+(.*?)(?:,|\.)', ato["descricao"], re.IGNORECASE)
            if match_prop:
                proprietario_atual = match_prop.group(1)
                data_consolidacao = ato["data"]
                break
    
    resumo = (
        f"Síntese da Matrícula Nº {dados['numero_matricula']}:\n"
        f"O imóvel localizado em {dados['dados_imovel'].get('endereco', 'Endereço não encontrado')}, "
        f"{dados['dados_imovel'].get('bairro', 'Bairro não encontrado')}, "
        f"pertence atualmente à {proprietario_atual}. "
        f"A propriedade foi consolidada em nome do credor em {data_consolidacao}. "
        f"A situação atual do imóvel é livre de ônus, conforme os últimos registros de cancelamento."
    )
    return resumo

# --- Execução do Pipeline ---
if __name__ == "__main__":
    # Coloque o nome do arquivo PDF que deseja processar
    # Supondo que os arquivos estão na mesma pasta que o script.
    # Se não estiverem, forneça o caminho completo. ex: "C:/Users/SeuUsuario/Docs/1444411417161.pdf"
    
    arquivos_pdf = ["1444411417161.pdf", "1444404404084.pdf"]

    for nome_arquivo in arquivos_pdf:
        print(f"--- Processando Matrícula: {nome_arquivo} ---")
        
        # Etapa 1: Extrair e limpar o texto
        texto_limpo = extrair_texto_limpo(f"../input/{nome_arquivo}")
        
        # Etapa 2: Segmentar e extrair para formato estruturado
        dados_estruturados = segmentar_e_extrair(texto_limpo)
        
        # Etapa 3: Gerar o resumo
        resumo_texto = gerar_resumo(dados_estruturados)
        
        # Etapa 4: Apresentar os resultados
        print("\n>>> RESUMO DA MATRÍCULA <<<\n")
        print(resumo_texto)
        
        print("\n>>> DADOS ESTRUTURADOS (JSON) <<<\n")
        # Usamos json.dumps para imprimir o dicionário de forma legível
        print(json.dumps(dados_estruturados, indent=2, ensure_ascii=False))
        
        print("\n" + "="*50 + "\n")

--- Processando Matrícula: 1444411417161.pdf ---

>>> RESUMO DA MATRÍCULA <<<

Síntese da Matrícula Nº 115751:
O imóvel localizado em Endereço não encontrado, Bairro não encontrado, pertence atualmente à CAIXA ECONÔMICA FEDERAL. A propriedade foi consolidada em nome do credor em Não encontrada. A situação atual do imóvel é livre de ônus, conforme os últimos registros de cancelamento.

>>> DADOS ESTRUTURADOS (JSON) <<<

{
  "numero_matricula": "115751",
  "dados_imovel": {
    "descricao_completa": "CERTIDÃO DE INTEIRO TEOR C E R T I F I C O, autorizado pela lei e a pedido verbal de pessoa interessada, que revendo a Matrícula 115.751 do Registro Geral do 2º Ofício do Registro de Imóveis Zona Norte) da Comarca desta Capital, de meu cargo, dele, sob o número de Ordem 115.751 desta data de 02 de outubro de 2014, consta Transcrição: UNIDADE AUTÔNOMA DE Nº 3102 DA TORRE B, NÃO AVERBADA, APENAS INCORPORADA, do Edifício Residencial que terá a denominação de EDIFÍCIO NEXT TOWERS, a ser edificad

In [40]:
a = extrair_texto_limpo("../input/1444411417161.pdf")

print(a)

CERTIDÃO DE INTEIRO TEOR C E R T I F I C O, autorizado pela lei e a pedido verbal de pessoa interessada, que revendo a Matrícula 115.751 do Registro Geral do 2º Ofício do Registro de Imóveis Zona Norte) da Comarca desta Capital, de meu cargo, dele, sob o número de Ordem 115.751 desta data de 02 de outubro de 2014, consta Transcrição: UNIDADE AUTÔNOMA DE Nº 3102 DA TORRE B, NÃO AVERBADA, APENAS INCORPORADA, do Edifício Residencial que terá a denominação de EDIFÍCIO NEXT TOWERS, a ser edificado no Lote de terreno próprio sob nº 107 da Quadra 435, situado a Rua Iracema Guedes Lins, no bairro Altiplano Cabo Branco, nesta cidade, que será composta de: varanda, sala de estar/jantar, 3 (três) quartos sendo um social , um suíte com varanda e um suíte reversível, wcb social reversível a suíte, cozinha/área de serviço e no pavimento de garagem duas vagas cobertas sendo uma confinada, localizadas no semi-subsolo, com área real privativa (Principal) 71,90m², área real privativa acessória (vaga de 